<a href="https://colab.research.google.com/github/sakshi21pawar/AI-intro/blob/main/shop_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
!pip install gradio pandas matplotlib scikit-learn seaborn

print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


In [ ]:
# Import all required libraries
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.svm import SVR
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
import warnings
import io
import base64
import os

# Suppress warnings
warnings.filterwarnings('ignore')

# Set style for better looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Define file paths
DATA_FILE = "shop_data.csv"
AI_ANALYSIS_IMG = "ai_analysis.png"
MONTH_REPORT_IMG = "month_report.png"
CATEGORY_IMG = "category_analysis.png"
FORECAST_IMG = "profit_forecast.png"

print("✅ Libraries imported and setup completed!")

✅ Libraries imported and setup completed!


In [ ]:
# Initialize sample dataset if it doesn't exist
def initialize_sample_data():
    if not os.path.exists(DATA_FILE):
        # Create sample data for the past 6 months
        dates = pd.date_range(start='2024-01-01', end='2024-06-30', freq='D')
        np.random.seed(42)

        records = []
        income_categories = ['Sales', 'Services', 'Online', 'Consulting']
        expense_categories = ['Inventory', 'Rent', 'Utilities', 'Salaries', 'Marketing', 'Supplies', 'Maintenance']

        for date in dates:
            # Add income records (1-3 per day)
            num_income = np.random.randint(1, 4)
            for _ in range(num_income):
                amount = np.random.normal(500, 200)
                records.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'type': 'Income',
                    'item': f"{np.random.choice(income_categories)} Transaction",
                    'amount': max(50, amount),
                    'category': np.random.choice(income_categories)
                })

            # Add expense records (1-5 per day)
            num_expense = np.random.randint(1, 6)
            for _ in range(num_expense):
                amount = np.random.normal(150, 100)
                records.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'type': 'Expense',
                    'item': f"{np.random.choice(expense_categories)} Payment",
                    'amount': max(10, amount),
                    'category': np.random.choice(expense_categories)
                })

        df = pd.DataFrame(records)
        df.to_csv(DATA_FILE, index=False)
        print(f"✅ Sample dataset created with {len(df)} records")
    else:
        print("✅ Existing dataset found")

initialize_sample_data()

✅ Sample dataset created with 912 records


In [ ]:
# Data handling functions
def load_data():
    """Load data from CSV file safely"""
    try:
        df = pd.read_csv(DATA_FILE)
        df['date'] = pd.to_datetime(df['date'])
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame(columns=['date', 'type', 'item', 'amount', 'category'])

def save_data(df):
    """Save data back to CSV"""
    try:
        df.to_csv(DATA_FILE, index=False)
        return True
    except Exception as e:
        print(f"Error saving data: {e}")
        return False

def get_monthly_summary():
    """Get monthly summary of income, expenses, and profit"""
    df = load_data()
    if df.empty:
        return pd.DataFrame()

    # Create month-year column for grouping
    df['month_year'] = df['date'].dt.strftime('%Y-%m')
    monthly = df.groupby(['month_year', 'type'])['amount'].sum().unstack(fill_value=0)
    monthly['Profit'] = monthly.get('Income', 0) - monthly.get('Expense', 0)
    return monthly

In [ ]:
# Record management functions
def add_record(date, record_type, item, amount, category):
    """Add a new financial record"""
    df = load_data()

    new_record = pd.DataFrame({
        'date': [pd.to_datetime(date)],
        'type': [record_type],
        'item': [item],
        'amount': [float(amount)],
        'category': [category]
    })

    df = pd.concat([df, new_record], ignore_index=True)
    save_data(df)
    return f"✅ Record added successfully! {record_type}: {item} - ${amount:.2f}"

def view_data():
    """View all data with formatting"""
    df = load_data()
    if df.empty:
        return "No data available. Please add some records first."

    # Format for display
    display_df = df.copy()
    display_df['date'] = display_df['date'].dt.strftime('%Y-%m-%d')
    display_df['amount'] = display_df['amount'].apply(lambda x: f"${x:.2f}")
    return display_df

def upload_file(file):
    """Handle CSV file upload"""
    try:
        uploaded_df = pd.read_csv(file.name)
        required_cols = ['date', 'type', 'item', 'amount', 'category']

        if all(col in uploaded_df.columns for col in required_cols):
            # Merge with existing data
            existing_df = load_data()
            combined_df = pd.concat([existing_df, uploaded_df], ignore_index=True)
            combined_df['date'] = pd.to_datetime(combined_df['date'])
            save_data(combined_df)
            return f"✅ File uploaded successfully! Added {len(uploaded_df)} records."
        else:
            return "❌ Invalid file format. Required columns: date, type, item, amount, category"
    except Exception as e:
        return f"❌ Error uploading file: {str(e)}"

def download_csv():
    """Provide CSV download"""
    df = load_data()
    if df.empty:
        # Return empty file path if no data
        return None

    # Save to a temporary file for download
    download_path = "shop_data_export.csv"
    df.to_csv(download_path, index=False)
    return download_path

In [ ]:
# AI/ML Analysis Functions
def ai_analysis():
    """Perform AI-based analysis with SVM and Isolation Forest"""
    df = load_data()
    if df.empty or len(df) < 10:
        return "Not enough data for analysis. Please add more records.", None

    # Prepare monthly data
    monthly = get_monthly_summary()
    if monthly.empty:
        return "No monthly data available for analysis.", None

    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Plot 1: Profit trend with SVM regression and anomaly detection
    months = list(range(len(monthly)))
    profits = monthly['Profit'].values

    # SVM Regression for trend
    if len(months) > 1:
        X = np.array(months).reshape(-1, 1)
        y = profits

        # Scale the data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # SVM Regression
        svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
        svr.fit(X_scaled, y)
        trend_line = svr.predict(X_scaled)

        # Anomaly detection using Isolation Forest
        iso_forest = IsolationForest(contamination=0.2, random_state=42)
        anomalies = iso_forest.fit_predict(y.reshape(-1, 1))

        # Plot profit trend
        ax1.plot(months, profits, 'o-', label='Actual Profit', linewidth=2, markersize=8)
        ax1.plot(months, trend_line, '--', label='SVM Trend', linewidth=2)

        # Mark anomalies
        anomaly_indices = [i for i, a in enumerate(anomalies) if a == -1]
        ax1.scatter(anomaly_indices, profits[anomaly_indices],
                   color='red', s=100, label='Anomalies', zorder=5)

        ax1.set_title('💰 Profit Trend Analysis with Anomaly Detection', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Month Index')
        ax1.set_ylabel('Profit ($)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Predict next month
        if len(months) > 2:
            next_month = len(months)
            next_month_scaled = scaler.transform([[next_month]])
            predicted_profit = svr.predict(next_month_scaled)[0]
    else:
        ax1.text(0.5, 0.5, 'Need more data\nfor analysis',
                ha='center', va='center', transform=ax1.transAxes, fontsize=12)
        predicted_profit = None
        anomaly_indices = []

    # Plot 2: Income vs Expense distribution
    total_income = df[df['type'] == 'Income']['amount'].sum()
    total_expense = df[df['type'] == 'Expense']['amount'].sum()

    labels = ['Income', 'Expenses']
    sizes = [total_income, total_expense]
    colors = ['#2ecc71', '#e74c3c']

    ax2.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax2.set_title('📊 Income vs Expenses Distribution', fontsize=14, fontweight='bold')

    plt.tight_layout()

    # Save the figure
    plt.savefig(AI_ANALYSIS_IMG, dpi=150, bbox_inches='tight')
    plt.close()

    # Generate text summary
    summary = f"""
    ## 🤖 AI Analysis Summary

    ### 📈 Financial Overview:
    - **Total Income**: ${total_income:,.2f}
    - **Total Expenses**: ${total_expense:,.2f}
    - **Net Profit**: ${total_income - total_expense:,.2f}
    - **Profit Margin**: {((total_income - total_expense) / total_income * 100) if total_income > 0 else 0:.1f}%

    ### 🔍 Trend Analysis:
    """

    if predicted_profit is not None:
        summary += f"- **Next Month Profit Prediction**: ${predicted_profit:,.2f}\n"
        trend_direction = "increasing" if predicted_profit > profits[-1] else "decreasing"
        summary += f"- **Trend**: {trend_direction.capitalize()}\n"

    summary += f"- **Anomalies Detected**: {len(anomaly_indices)} unusual profit months\n"

    if len(anomaly_indices) > 0:
        summary += "\n### ⚠️ Attention Needed:\n"
        summary += "Some months show unusual profit patterns. Consider reviewing those periods.\n"

    return summary, AI_ANALYSIS_IMG

In [ ]:
# Monthly Report Functions
def generate_monthly_report(month_input):
    """Generate monthly report with visualization"""
    df = load_data()
    if df.empty:
        return "No data available. Please add records first.", None, None

    try:
        # Create month_year column for filtering
        df['month_year'] = df['date'].dt.strftime('%Y-%m')

        # Check available months
        available_months = sorted(df['month_year'].unique())

        # Filter data for the selected month
        month_data = df[df['month_year'] == month_input]

        if month_data.empty:
            return f"❌ No data available for {month_input}. Available months: {', '.join(available_months)}", None, None

        # Calculate totals
        income = month_data[month_data['type'] == 'Income']['amount'].sum()
        expense = month_data[month_data['type'] == 'Expense']['amount'].sum()
        profit = income - expense

        # Create visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

        # Pie chart for income vs expenses
        labels = ['Income', 'Expenses']
        sizes = [income, expense]
        colors = ['#2ecc71', '#e74c3c']

        ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        ax1.set_title(f'📊 {month_input} - Income vs Expenses', fontweight='bold')

        # Bar chart for detailed view
        categories = ['Income', 'Expenses', 'Profit']
        values = [income, expense, profit]
        bar_colors = ['#2ecc71', '#e74c3c', '#3498db']

        bars = ax2.bar(categories, values, color=bar_colors, alpha=0.8)
        ax2.set_title(f'📈 {month_input} - Financial Summary', fontweight='bold')
        ax2.set_ylabel('Amount ($)')

        # Add value labels on bars
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + max(values)*0.01,
                    f'${value:,.0f}', ha='center', va='bottom', fontweight='bold')

        plt.tight_layout()
        plt.savefig(MONTH_REPORT_IMG, dpi=150, bbox_inches='tight')
        plt.close()

        # Generate HTML report
        html_report = generate_html_report(month_input, income, expense, profit)

        # Calculate growth compared to previous month
        growth_percentage = calculate_monthly_growth(df, month_input)

        # Text summary
        text_summary = f"""
        ## 📅 Monthly Report - {month_input}

        ### 💰 Financial Summary:
        - **Total Income**: ${income:,.2f}
        - **Total Expenses**: ${expense:,.2f}
        - **Net Profit**: ${profit:,.2f}
        - **Profit Margin**: {(profit/income*100) if income > 0 else 0:.1f}%

        ### 📊 Performance Indicators:
        - **Expense Ratio**: {(expense/income*100) if income > 0 else 0:.1f}% of income
        - **Monthly Growth**: {growth_percentage:.1f}%
        - **Transactions**: {len(month_data)} records

        ### 📈 Business Insight:
        {get_business_insight(income, expense, profit)}
        """

        return text_summary, MONTH_REPORT_IMG, html_report

    except Exception as e:
        return f"❌ Error generating report: {str(e)}", None, None

def calculate_monthly_growth(df, current_month_str):
    """Calculate month-over-month growth"""
    try:
        # Get all unique months from data
        all_months = sorted(df['month_year'].unique())
        current_index = all_months.index(current_month_str)

        if current_index > 0:
            prev_month = all_months[current_index - 1]

            # Calculate profit for current and previous month
            current_data = df[df['month_year'] == current_month_str]
            current_income = current_data[current_data['type'] == 'Income']['amount'].sum()
            current_expense = current_data[current_data['type'] == 'Expense']['amount'].sum()
            current_net = current_income - current_expense

            prev_data = df[df['month_year'] == prev_month]
            prev_income = prev_data[prev_data['type'] == 'Income']['amount'].sum()
            prev_expense = prev_data[prev_data['type'] == 'Expense']['amount'].sum()
            prev_net = prev_income - prev_expense

            if prev_net != 0:
                return ((current_net - prev_net) / abs(prev_net)) * 100
        return 0
    except:
        return 0

def get_business_insight(income, expense, profit):
    """Generate business insight based on financials"""
    if income == 0:
        return "No income recorded this month. Focus on generating revenue."

    profit_margin = (profit / income) * 100
    expense_ratio = (expense / income) * 100

    if profit_margin > 20:
        return "🎉 Excellent profitability! Consider reinvesting in growth."
    elif profit_margin > 10:
        return "👍 Good performance. Maintain current operations."
    elif profit_margin > 0:
        return "⚠️  Low profit margin. Look for cost-saving opportunities."
    else:
        return "🚨 Operating at a loss. Urgent review of expenses needed."

def generate_html_report(month, income, expense, profit):
    """Generate downloadable HTML report"""
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Business Report - {month}</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            .header {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                     color: white; padding: 30px; border-radius: 10px; text-align: center; }}
            .metrics {{ display: flex; justify-content: space-around; margin: 30px 0; }}
            .metric {{ text-align: center; padding: 20px; border-radius: 10px;
                      box-shadow: 0 4px 6px rgba(0,0,0,0.1); }}
            .income {{ background: #e8f5e8; border-left: 4px solid #2ecc71; }}
            .expense {{ background: #ffe8e8; border-left: 4px solid #e74c3c; }}
            .profit {{ background: #e8f4fd; border-left: 4px solid #3498db; }}
            .value {{ font-size: 24px; font-weight: bold; margin: 10px 0; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>💼 Smart Shop Tracker - Monthly Report</h1>
            <h2>{month}</h2>
        </div>

        <div class="metrics">
            <div class="metric income">
                <h3>💰 Total Income</h3>
                <div class="value">${income:,.2f}</div>
            </div>
            <div class="metric expense">
                <h3>💸 Total Expenses</h3>
                <div class="value">${expense:,.2f}</div>
            </div>
            <div class="metric profit">
                <h3>📈 Net Profit</h3>
                <div class="value">${profit:,.2f}</div>
            </div>
        </div>

        <div style="text-align: center; margin-top: 30px;">
            <p><strong>Profit Margin:</strong> {(profit/income*100) if income > 0 else 0:.1f}%</p>
            <p><strong>Generated on:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M')}</p>
        </div>
    </body>
    </html>
    """

    # Save HTML file
    with open("monthly_report.html", "w") as f:
        f.write(html_content)

    return html_content

In [ ]:
# Category Analysis Functions
def category_summary():
    """Generate category-wise spending analysis"""
    df = load_data()
    if df.empty:
        return "No data available.", None

    # Expense analysis by category
    expense_df = df[df['type'] == 'Expense']
    if expense_df.empty:
        return "No expense data available.", None

    category_totals = expense_df.groupby('category')['amount'].sum().sort_values(ascending=False)

    # Take top 10 categories
    top_categories = category_totals.head(10)

    # Create visualization
    fig, ax = plt.subplots(figsize=(12, 8))

    colors = plt.cm.Set3(np.linspace(0, 1, len(top_categories)))
    bars = ax.barh(range(len(top_categories)), top_categories.values, color=colors)

    ax.set_yticks(range(len(top_categories)))
    ax.set_yticklabels(top_categories.index)
    ax.set_xlabel('Amount ($)')
    ax.set_title('📊 Top 10 Spending Categories', fontsize=16, fontweight='bold')

    # Add value labels
    for i, (category, value) in enumerate(zip(top_categories.index, top_categories.values)):
        ax.text(value + max(top_categories.values) * 0.01, i,
                f'${value:,.0f}', va='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig(CATEGORY_IMG, dpi=150, bbox_inches='tight')
    plt.close()

    # Generate markdown summary
    summary = "## 🏷️ Category Spending Analysis\n\n"
    summary += "### Top Spending Categories:\n"

    for i, (category, amount) in enumerate(top_categories.items(), 1):
        percentage = (amount / category_totals.sum()) * 100
        summary += f"{i}. **{category}**: ${amount:,.2f} ({percentage:.1f}%)\n"

    summary += f"\n**Total Expenses**: ${category_totals.sum():,.2f}\n"

    # Identify highest spending category
    top_category = top_categories.index[0]
    top_amount = top_categories.values[0]
    summary += f"\n### 💡 Insight:\n"
    summary += f"Your highest spending is in **{top_category}** (${top_amount:,.2f})"

    return summary, CATEGORY_IMG

In [ ]:
# Advanced Features
def advanced_analysis():
    """Perform advanced analysis with forecasting and AI suggestions"""
    df = load_data()
    if df.empty or len(df) < 3:
        return "Not enough data for advanced analysis.", None, "Add more data to unlock AI insights."

    monthly = get_monthly_summary()
    if len(monthly) < 3:
        return "Need at least 3 months of data for forecasting.", None, "Continue tracking to get predictions."

    # Prepare data for forecasting
    months = list(range(len(monthly)))
    profits = monthly['Profit'].values

    # Linear Regression for forecasting
    X = np.array(months).reshape(-1, 1)
    y = profits

    lr = LinearRegression()
    lr.fit(X, y)

    # Forecast next 3 months
    future_months = np.array([len(months), len(months)+1, len(months)+2]).reshape(-1, 1)
    forecasts = lr.predict(future_months)

    # Create forecast visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Plot 1: Historical profit with forecast
    all_months = list(months) + [len(months), len(months)+1, len(months)+2]
    all_profits = list(profits) + list(forecasts)

    ax1.plot(months, profits, 'o-', label='Historical', linewidth=2, markersize=8)
    ax1.plot(future_months, forecasts, 's--', color='orange',
             label='Forecast', linewidth=2, markersize=8)
    ax1.fill_between(future_months.flatten(), forecasts * 0.8, forecasts * 1.2,
                     alpha=0.2, color='orange', label='Forecast Range')

    ax1.set_title('📈 Profit Forecast (Next 3 Months)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Month Index')
    ax1.set_ylabel('Profit ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Business health indicators
    total_income = df[df['type'] == 'Income']['amount'].sum()
    total_expense = df[df['type'] == 'Expense']['amount'].sum()

    # Calculate health score (0-100)
    if total_income > 0:
        profit_margin = (total_income - total_expense) / total_income
        health_score = min(100, max(0, profit_margin * 100 + 50))

        # Additional factors
        expense_ratio = total_expense / total_income if total_income > 0 else 1
        growth_trend = calculate_growth_trend(profits)
        health_score += growth_trend * 10
        health_score = min(100, max(0, health_score))
    else:
        health_score = 0

    # Health gauge
    categories = ['Profit Margin', 'Growth Trend', 'Stability']
    scores = [min(100, profit_margin * 100), min(100, growth_trend * 50 + 50), 75]
    colors = ['green' if s > 70 else 'orange' if s > 40 else 'red' for s in scores]

    bars = ax2.bar(categories, scores, color=colors, alpha=0.7)
    ax2.set_ylim(0, 100)
    ax2.set_title('🏥 Business Health Score', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Score (0-100)')

    # Add value labels
    for bar, score in zip(bars, scores):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 2,
                f'{score:.0f}', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.savefig(FORECAST_IMG, dpi=150, bbox_inches='tight')
    plt.close()

    # Generate AI suggestions
    ai_suggestions = generate_ai_suggestions(df, monthly)

    # Summary text
    summary = f"""
    ## 🚀 Advanced Business Insights

    ### 📊 Financial Forecast:
    - **Next Month**: ${forecasts[0]:,.2f}
    - **Month +2**: ${forecasts[1]:,.2f}
    - **Month +3**: ${forecasts[2]:,.2f}

    ### 🏥 Business Health:
    - **Overall Health Score**: {health_score:.0f}/100
    - **Profit Margin**: {profit_margin*100:.1f}%
    - **Growth Trend**: {'📈 Positive' if growth_trend > 0 else '📉 Negative'}

    ### 💡 AI Recommendations:
    {ai_suggestions}
    """

    return summary, FORECAST_IMG, ai_suggestions

def calculate_growth_trend(profits):
    """Calculate overall growth trend"""
    if len(profits) < 2:
        return 0
    return (profits[-1] - profits[0]) / abs(profits[0]) if profits[0] != 0 else 0

def generate_ai_suggestions(df, monthly_data):
    """Generate AI-powered business suggestions"""
    suggestions = []

    # Analyze spending patterns
    expense_by_category = df[df['type'] == 'Expense'].groupby('category')['amount'].sum()
    if not expense_by_category.empty:
        top_spending_category = expense_by_category.idxmax()
        top_spending_amount = expense_by_category.max()
        total_expenses = expense_by_category.sum()

        if top_spending_amount / total_expenses > 0.4:  # If one category is >40% of expenses
            suggestions.append(f"• **Reduce spending in {top_spending_category}** - it's {top_spending_amount/total_expenses*100:.1f}% of total expenses")

    # Analyze profit trends
    if len(monthly_data) > 1:
        recent_profit = monthly_data['Profit'].iloc[-1]
        avg_profit = monthly_data['Profit'].mean()

        if recent_profit < avg_profit * 0.7:
            suggestions.append("• **Focus on profitability** - recent profits are below average")
        elif recent_profit > avg_profit * 1.3:
            suggestions.append("• **Great work!** Consider reinvesting surplus profits")

    # General suggestions based on data volume
    total_records = len(df)
    if total_records < 50:
        suggestions.append("• **Track more transactions** to get better insights")
    else:
        suggestions.append("• **Excellent data tracking!** Continue monitoring for trends")

    # Add seasonal suggestion
    current_month = datetime.now().month
    if current_month in [11, 12]:  # November/December
        suggestions.append("• **Holiday season** - Plan for increased sales and inventory")
    elif current_month in [1, 2]:  # January/February
        suggestions.append("• **Post-holiday period** - Review Q4 performance and set annual goals")

    return "\n".join(suggestions) if suggestions else "• Continue current practices - your business is on track!"

In [ ]:
# Gradio Dashboard UI
def create_dashboard():
    """Create the main Gradio dashboard interface"""

    with gr.Blocks(theme=gr.themes.Soft(), title="💼 Smart Shop Tracker") as demo:
        gr.Markdown(
            """
            # 💼 Smart Shop Tracker (Enhanced ML Edition)
            ### *AI-Powered Financial Analytics for Small Businesses*

            Welcome to your intelligent business management dashboard! Track income, expenses,
            and get AI-driven insights to grow your business.
            """
        )

        with gr.Tab("➕ Add Record"):
            gr.Markdown("## 📥 Add New Financial Record")
            with gr.Row():
                with gr.Column():
                    record_date = gr.Textbox(label="📅 Date (YYYY-MM-DD)", value=datetime.now().strftime("%Y-%m-%d"))
                    record_type = gr.Radio(["Income", "Expense"], label="💰 Type", value="Income")
                    item_name = gr.Textbox(label="📝 Item Description", placeholder="e.g., Product Sales, Rent Payment")
                    amount = gr.Number(label="💵 Amount ($)", value=0.0)
                    category = gr.Textbox(label="🏷️ Category", placeholder="e.g., Sales, Rent, Supplies")
                    add_btn = gr.Button("💾 Add Record", variant="primary")

                with gr.Column():
                    output_msg = gr.Textbox(label="Status", interactive=False)

            add_btn.click(
                add_record,
                inputs=[record_date, record_type, item_name, amount, category],
                outputs=[output_msg]
            )

        with gr.Tab("📋 View Data"):
            gr.Markdown("## 📊 All Financial Records")
            data_display = gr.Dataframe(
                label="Financial Data",
                headers=["Date", "Type", "Item", "Amount", "Category"],
                interactive=False,
                datatype=["str", "str", "str", "str", "str"]
            )
            with gr.Row():
                refresh_btn = gr.Button("🔄 Refresh Data")
                download_btn = gr.DownloadButton("📥 Download CSV")

            refresh_btn.click(view_data, outputs=[data_display])
            download_btn.click(download_csv, inputs=[], outputs=[download_btn])

        with gr.Tab("📈 AI-Based Analysis"):
            gr.Markdown("## 🤖 AI-Powered Financial Analysis")
            with gr.Row():
                analyze_btn = gr.Button("🚀 Run AI Analysis", variant="primary", size="lg")

            with gr.Row():
                with gr.Column():
                    analysis_output = gr.Markdown(label="Analysis Results")
                with gr.Column():
                    analysis_plot = gr.Image(label="Financial Analysis Chart")

            analyze_btn.click(ai_analysis, outputs=[analysis_output, analysis_plot])

        with gr.Tab("📅 Monthly Report"):
            gr.Markdown("## 📊 Monthly Financial Report")
            with gr.Row():
                month_input = gr.Textbox(
                    label="Select Month (YYYY-MM)",
                    value=datetime.now().strftime("%Y-%m"),
                    placeholder="e.g., 2024-01"
                )
                generate_btn = gr.Button("📈 Generate Report", variant="primary")

            with gr.Row():
                with gr.Column():
                    report_output = gr.Markdown(label="Monthly Summary")
                with gr.Column():
                    report_plot = gr.Image(label="Monthly Charts")

            html_download = gr.File(label="📄 Download HTML Report")

            generate_btn.click(
                generate_monthly_report,
                inputs=[month_input],
                outputs=[report_output, report_plot, html_download]
            )

        with gr.Tab("📊 Category Summary"):
            gr.Markdown("## 🏷️ Spending Analysis by Category")
            with gr.Row():
                category_btn = gr.Button("📊 Analyze Categories", variant="primary")

            with gr.Row():
                with gr.Column():
                    category_output = gr.Markdown(label="Category Analysis")
                with gr.Column():
                    category_plot = gr.Image(label="Top Categories Chart")

            category_btn.click(category_summary, outputs=[category_output, category_plot])

        with gr.Tab("📤 Upload CSV"):
            gr.Markdown("## 📁 Upload Financial Data")
            gr.Markdown("Upload a CSV file with columns: `date`, `type`, `item`, `amount`, `category`")

            file_upload = gr.File(
                label="Select CSV File",
                file_types=[".csv"],
                type="filepath"
            )
            upload_status = gr.Textbox(label="Upload Status", interactive=False)

            file_upload.upload(upload_file, inputs=[file_upload], outputs=[upload_status])

        with gr.Tab("💡 Forecast & AI Tips"):
            gr.Markdown("## 🔮 Advanced Forecasting & AI Recommendations")
            with gr.Row():
                advanced_btn = gr.Button("🎯 Get Advanced Insights", variant="primary", size="lg")

            with gr.Row():
                with gr.Column():
                    advanced_output = gr.Markdown(label="Advanced Analysis")
                with gr.Column():
                    advanced_plot = gr.Image(label="Forecast & Health Charts")

            with gr.Row():
                ai_tips = gr.Textbox(
                    label="🤖 AI Business Tips",
                    interactive=False,
                    lines=5,
                    max_lines=10
                )

            advanced_btn.click(
                advanced_analysis,
                outputs=[advanced_output, advanced_plot, ai_tips]
            )

        # Initialize data display
        demo.load(view_data, outputs=[data_display])

    return demo

In [ ]:
# Launch the application
print("🚀 Starting Smart Shop Tracker Dashboard...")

# Create and launch the dashboard
demo = create_dashboard()

# For Google Colab, we need to use share=True
try:
    demo.launch(share=True, debug=True)
    print("✅ Dashboard launched successfully!")
    print("📱 Your Smart Shop Tracker is now running!")
    print("💡 The app will automatically generate sample data if no existing data is found.")
except Exception as e:
    print(f"Note: {str(e)}")
    demo.launch(debug=True)
    print("✅ Dashboard launched successfully!")

🚀 Starting Smart Shop Tracker Dashboard...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://98ef63e72dd6a35cfb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2127, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1904, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/components/file.py", line 227, in postprocess
